In [1]:
import pandas as pd
# 정적크롤링
from bs4 import BeautifulSoup
import requests
import re
# 동적크롤링
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

In [2]:
programmers_res = pd.DataFrame(columns=['기업명','사원수','주소','직무','이용하는 기술스택','자격요건','직급','우대사항','해당 페이지 URL','기업 홈페이지 URL'])

In [3]:
# 홈페이지 접속
url = 'https://career.programmers.co.kr/job?page=1&tags=AWS%20EC2&order=recent'

driver = webdriver.Chrome()
driver.get(url)

while True:
    for corp in range(1,21):
        try:
            driver.find_element(By.XPATH, f'//*[@id="list-positions-wrapper"]/ul/li[{corp}]/div[2]/div[1]/h5/a').click()
            driver.switch_to.window(driver.window_handles[-1])
            time.sleep(1)

            soup = BeautifulSoup(driver.page_source, 'html.parser')
            time.sleep(1)

            # 1. 기업명
            corp_name = driver.find_element(By.XPATH, '//*[@id="career-app-legacy"]/div/div[1]/div[1]/header/div/div[2]/h4/a').text
            # 7. 주소
            address = driver.find_element(By.XPATH, '//*[@id="career-app-legacy"]/div/div[1]/div[1]/section/div/div[2]/div/div[2]').text
            # 8. 직무
            job_description = driver.find_element(By.XPATH, '//*[@id="career-app-legacy"]/div/div[1]/div[1]/section/div/div[1]/div[1]/div[2]').text
            # 9. 이용하는 기술 스택
            job_skill = driver.find_element(By.XPATH, '//*[@id="career-app-legacy"]/div/div[1]/div[1]/div[2]/section/ul').text
            # 10. 자격요건
            job_requirements = driver.find_element(By.XPATH, '//*[@id="career-app-legacy"]/div/div[1]/div[1]/div[3]/section[2]/div').text
            # 11. 직급
            career = driver.find_element(By.XPATH, '//*[@id="career-app-legacy"]/div/div[1]/div[1]/section/div/div[1]/div[4]/div[2]').text
            # 12. 우대사항
            job_preferential = driver.find_element(By.XPATH, '//*[@id="career-app-legacy"]/div/div[1]/div[1]/div[3]/section[3]/div/div').text
            # 13. 해당페이지 URL
            employment_url = soup.select('meta')[10].get('content')

            # 3,14번 크롤링을 위한 기업 클릭
            driver.find_element(By.XPATH, '//*[@id="career-app-legacy"]/div/div[1]/div[1]/header/div/div[2]/h4/a').click()
            driver.switch_to.window(driver.window_handles[-1])
            time.sleep(1)

            soup = BeautifulSoup(driver.page_source, 'html.parser')

            # 3. 사원 수
            employee = driver.find_element(By.XPATH, '//*[@id="career-app-legacy"]/div/div/div/div[2]/section[1]/ul/li[2]/h6[2]').text
            # 14. 기업 홈페이지 URL
            corp_url = soup.select('h6.list-value a')[0].get('href')

            programmers_temp = pd.DataFrame(
                [corp_name,employee,address,job_description,job_skill,job_requirements,career,job_preferential,
                employment_url,corp_url], 
                index = ['기업명','사원수','주소','직무','이용하는 기술스택','자격요건','직급','우대사항','해당 페이지 URL','기업 홈페이지 URL']).T
            
            programmers_res = pd.concat([programmers_res, programmers_temp])
            
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
            time.sleep(1)
        except:
            break
    
    # 페이지 반복
    try:
        driver.find_element(By.XPATH, '//*[@id="tab_position"]/div[3]/ul/li[5]/span').click()
        time.sleep(1)

    except:
        break

driver.quit()

In [8]:
programmers_res.to_csv('./programmers.csv')